In [1]:
# Тюнинг
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder, OneHotEncoder
from sklearn.preprocessing import RobustScaler

In [2]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
test_df = pd.read_csv(TEST_DATASET)

# first_negatives = train_df[train_df['y'] == 0][:train_df[train_df['y'] == 1]['y'].count()]
# train_df = pd.concat([train_df[train_df['y'] == 1], first_negatives])

# def random_undersample(df):
#     neg_count, pos_count = np.bincount(df['y'])
#     pos_df = df[df['y'] == 1]
#     neg_df = df[df['y'] == 0]
#     neg_df = neg_df.sample(n=pos_count, random_state=1708)
#     return pd.concat([pos_df, neg_df])

# balanced_df = random_undersample(train_df)

X, y = balanced_df.iloc[:, :-1], balanced_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

FEATURES_TO_DROP = [
    'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
    'Количество', 'Категорийный менеджер', 'Завод', 'Материал',
    'Отмена полного деблокирования заказа на закупку', 'Количество циклов согласования'
    ]

# FEATURES_TO_KEEP = ['Поставщик', 'Длительность', 'Закупочная организация', 'Балансовая единица',
#                     'ЕИ', 'Сумма', 'Вариант поставки', 'НРП', 'Вариант поставки',
#                     'Месяц1', 'Месяц2', 'Месяц3', 'День недели 2']
# FEATURES_TO_DROP = X.columns[~X.columns.isin(FEATURES_TO_KEEP)]

X_train = X_train.drop(FEATURES_TO_DROP, axis=1)
X_test = X_test.drop(FEATURES_TO_DROP, axis=1)
test_df = test_df.drop(FEATURES_TO_DROP, axis=1)

In [3]:
cat_features = [
    'Закупочная организация', 'Поставщик',
    'Балансовая единица', 'ЕИ', 'Вариант поставки'
    ]

In [4]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, transform_train=True):
        self.transform_train = transform_train
        self.cat_features = cat_features

        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.onehot_encoder = OneHotEncoder(cols=['Поставщик'])
        self.robust_scaler = RobustScaler()

    def fit(self, X, y=None):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Месяц1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Месяц1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Месяц2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Месяц2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Месяц3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Месяц3'] / 12)

        # Категориальные фичи
        X_ = self.bin_encoder.fit_transform(X_)

        features_to_drop = [
            'День недели 2', 'Месяц1', 'Месяц2', 'Месяц3'
            ]

        X_ = X_.drop(features_to_drop, axis=1)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        # self.robust_scaler.fit(X_)

        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Месяц1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Месяц1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Месяц2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Месяц2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Месяц3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Месяц3'] / 12)

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        features_to_drop = [
            'День недели 2', 'Месяц1', 'Месяц2', 'Месяц3'
            ]
            
        X_ = X_.drop(features_to_drop, axis=1)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        # X_ = self.robust_scaler.transform(X_)

        return X_


In [ ]:
# data_preprocessor.fit(X_train)
# X_train_preproc = data_preprocessor.transform(X_train)
# X_test_preproc = data_preprocessor.transform(X_test)

In [5]:
# Функция оптимизации
def objective(trial: opt.Trial):
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.1, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 700, 1500, 50)
    max_depth = trial.suggest_int('max_depth', 6, 16)
    max_bin = trial.suggest_int('max_bin', 64, 352),
    num_leaves = trial.suggest_int('num_leaves', 64, 256)
    reg_lambda = trial.suggest_float('l2_reg', 0.1, 1, log=True)

    # Модель
    data_preprocessor = DataPreprocessor(cat_features)
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        reg_lambda=reg_lambda,
        max_bin=max_bin,
        n_jobs=-1,
        force_col_wise=True
    )

    rfe = RFE(estimator=model)

    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])
    
    cv_score = cross_val_score(pipeline, X_train, y_train, cv=StratifiedKFold(n_splits=5), scoring='f1_macro', n_jobs=-1)
    accuracy = cv_score.mean()

    return accuracy

In [ ]:
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

In [47]:
best_params = {
    'learning_rate': 0.3, 
    'n_estimators': 1000, 
    'max_depth': 10, 
    'max_bin': 128, 
    'num_leaves': 128, 
    'reg_lambda': 0.2,
    }

# Модель
data_preprocessor = DataPreprocessor(cat_features)
model = LGBMClassifier(
    **best_params,
    n_jobs=-1,
    force_col_wise=True,
    # is_unbalance=True
)

pipeline = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

In [48]:
pipeline.fit(X_train, y_train)
f1_score(y_test, pipeline.predict(X_test), average='macro')

[LightGBM] [Info] Number of positive: 116489, number of negative: 173320
[LightGBM] [Info] Total Bins 1412
[LightGBM] [Info] Number of data points in the train set: 289809, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.401951 -> initscore=-0.397343
[LightGBM] [Info] Start training from score -0.397343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


0.9835016910525334

In [49]:
preds = pipeline.predict(test_df)
submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})

In [50]:
submit_df.to_csv('submission.csv', index=False)

In [ ]:
# Модель
X_ = X_train.copy()
X_.columns = [num for num in range(0, len(X_train.columns))]

model = LGBMClassifier(
    **best_params,
    n_jobs=-1,
    force_col_wise=True,
)

model.fit(X_train, y_train)

X_

In [ ]:
pipeline['model'].feature_importances_